# Mask R-CNN - Visualize Feature Maps / Activations

Visualize feature maps (activations) and filters of Mask R-CNN model.

In [ ]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Root directory of the project
ROOT_DIR = "C:\\Users\\Martin\\Documents\\LEGOFinder\\Keras\\lego_object_detection\\maskrcnn"

%load_ext autoreload
%autoreload 2

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log

from keras.utils.vis_utils import plot_model

# Imports for keras-vis
from keras import activations
from vis.visualization import visualize_activation, get_num_filters
from vis.input_modifiers import Jitter
from vis.utils import utils

from samples.lego import lego

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")


In [ ]:
# Path to Lego trained weights
#LEGO_WEIGHTS_PATH = "C:/Users/Martin/Documents/LEGOFinder/Keras/lego_object_detection/maskrcnn/snapshots/mask_rcnn_lego_0025.h5" # Referenz Gewichte mit guten Resultaten
LEGO_WEIGHTS_PATH = "C:/Users/Martin/Google Drive/Colab/maskrcnn/snapshots/lego20200605T0653/mask_rcnn_lego_0040.h5"

# load either "train", "val" or "eval"
DATASET = "val"

## Configurations

In [ ]:
config = lego.LegoConfig()
LEGO_DIR = os.path.join(ROOT_DIR, "datasets", "lego")

In [ ]:
# Override the training configurations with a few
# changes for inferencing.
class InferenceConfig(config.__class__):
    # Run detection on one image at a time
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    DETECTION_MIN_CONFIDENCE = 0.8
    USE_RPN_ROIS = True
    USE_RANDOM_RPN_ROIS = False
    USE_STAGE_TWO = False
    BACKBONE = "resnet152"
    PLOT_GRAPH = True

config = InferenceConfig()
config.display()

## Notebook Preferences

In [ ]:
# Device to load the neural network on. Useful if you're training a model on the same machine, in which case use CPU and leave the GPU for training.
DEVICE = "/cpu:0"  # /cpu:0 or /gpu:0

# Inspect the model in training or inference modes values: 'inference' or 'training' TODO: code for 'training' test mode not ready yet
TEST_MODE = "inference"

In [ ]:
def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

## Load Dataset

In [ ]:
# Load validation dataset
dataset = lego.LegoDataset()
dataset.load_lego(LEGO_DIR, DATASET)

# Must call before using the dataset
dataset.prepare()

print("Images: {}\nClasses: {}".format(len(dataset.image_ids), dataset.class_names))

## Load Model

In [ ]:
# Create model in inference mode
with tf.device(DEVICE):
    model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# print model
#model.keras_model.summary()

# show final model graph
#if config.PLOT_GRAPH:
#    plot_model(model.keras_model, to_file='maskrcnn_graph.png', show_shapes=True, show_layer_names=True)

In [ ]:
# Set path to lego weights file

# Load weights
print("Loading weights ", LEGO_WEIGHTS_PATH)
model.load_weights(LEGO_WEIGHTS_PATH, by_name=True)

In [ ]:
# Choose image to analyse
if DATASET == "eval":
    image_id = dataset.get_image_id("0000000001.png") # if eval set us the image with the Lego haufen
else:
    image_ids = np.random.choice(dataset.image_ids, 1)
    image_id = image_ids[0]
    
image, image_meta, gt_class_ids, gt_bboxes, gt_masks = modellib.load_image_gt(dataset, config, image_id, use_mini_mask=False)

## Visualize Feature Maps (Activations)

In some cases it helps to look at the output from different layers and visualize them to catch issues and odd patterns.

In [ ]:
# Get activations of a few sample layers from top level mrcnn model
if config.USE_RPN_ROIS:
    # With keras-resnet as backbone
    mrcnn_model_layer_names  = ['input_image', 'conv1', 'conv1_relu', 'res2c_relu', 'res3d_relu', 'res4f_relu', 'fpn_p6', 'fpn_p4', 'fpn_p3']

    # If native Mask R-CNN is used as backbone
    # activation_layer_names = ['input_image', 'conv1', 'activation_1', 'res2c_out', 'res3c_out', 'res4w_out', 'fpn_p6', 'fpn_p4']
else:
    mrcnn_model_layer_names  = ['input_image', 'conv1', 'conv1_relu', 'res2c_relu', 'res3d_relu', 'res4f_relu', 'fpn_p6', 'fpn_p4', 'input_norm_classifier']

# Get layers of backbone
outputs = [(layer, tf.identity(model.keras_model.get_layer(layer).output)) for layer in mrcnn_model_layer_names]

if config.USE_RPN_ROIS:
    activations = model.run_graph([image], outputs, config)
else:
    activations = model.run_graph([image], outputs, config, gt_class_ids, gt_bboxes, gt_masks)


In [ ]:
# Input image (normalized)
_ = plt.imshow(modellib.unmold_image(activations["input_image"][0],config))

In [ ]:
# Show backbone feature map
max_maps = 10

for name in mrcnn_model_layer_names:
    d = activations[name].shape[-1]
    dd = d if d <= max_maps else max_maps
    titles = [name + '_' + str(i) for i in range(dd)]
    display_images(np.transpose(activations[name][0,:,:,:dd], [2, 0, 1]), cols=max_maps, titles=titles, figsize=20, fontsize=12)

## Visualize Convnet Filters

Now we will visualize the main building block of a CNN, the filters. There is one catch though, we won’t actually visualize the filters themselves, but instead we will display the patterns each filter maximally responds to. Remember that the filters are of size 3x3 meaning they have the height and width of 3 pixels, pretty small. So as a proxy to visualizing a filter, we will generate an input image where this filter activates the most. We will visualize filters at the last layer of each convolution block. To clear any confusion, in the previous section we visualized the feature maps, the output of the convolution operation. Now we are visualizing the filters, the main structure used in the convolution operation.

In [ ]:
# Visualize filters using custom function
from mrcnn import visualize

layer_name = 'conv1'
layer_name = 'res3a_branch2a'
stitched_filter_images = visualize.visualize_layer(model.keras_model, layer_name, filter_range=(0, None), grid_dimensions=None)

plt.figure(figsize=(20, 30))
plt.title(layer_name)
plt.axis('off')
plt.imshow(stitched_filter_images)
plt.show()

# EXPERIMENTAL

In [ ]:
# Do same for RPN model

outputs = []

# Find RPN model
for layer in model.keras_model.layers:
    # Is the layer a model?
    if layer.__class__.__name__ == 'Model':
        rpn_model = layer
        print("Found RPN model.")

# Get layers of backbone
outputs.append(('input_image', tf.identity(model.keras_model.get_layer('input_image').output)))
outputs.append(('conv1', tf.identity(model.keras_model.get_layer('conv1').output)))
outputs.append(('rpn_class_raw', tf.identity(rpn_model.get_layer('rpn_class_raw').output)))
outputs.append(('rpn_bbox_pred', tf.identity(rpn_model.get_layer('rpn_bbox_pred').output)))

print(outputs)
activations = model.run_graph([image], outputs, config)


In [ ]:
# To debug and install keras-viz do:
#
# conda activate maskrcnn
# cd C:\Users\Martin\Documents\LEGOFinder\Software\keras-vis-master>
# python setup.py install --record files.txt

layer_idx = utils.find_layer_idx(model.keras_model, 'conv1')

img = visualize_activation(model.keras_model, layer_idx, filter_indices=10, tv_weight=0., input_modifiers=[Jitter(0.05)], max_iter=300)

In [ ]:
from keras import activations
from vis.visualization import visualize_activation, get_num_filters
from vis.input_modifiers import Jitter
from vis.utils import utils


max_filters = 2
selected_indices = []
vis_images = [[], [], [], [], []]
i = 0

# Specify certain filter index if you want manually
selected_filters = [[0, 3, 11, 25, 26, 33, 42, 62], 
                    [8, 21, 23, 38, 39, 45, 50, 79], 
                    [40, 48, 52, 54, 81, 107, 224, 226],
                    [58, 79, 86, 216, 307, 426, 497, 509],
                    [2, 7, 41, 84, 103, 306, 461, 487]]

selected_filters = 0

conv_layer_names        = ['conv1']

for layer_name in conv_layer_names:
    layer_idx = utils.find_layer_idx(model.keras_model, layer_name)

    # Visualize all filters in this layer.
    if selected_filters:
        filter_indices = selected_filters[i]
 
    else:
        # Select random filters / kernels, but not more than max_filters
        n = get_num_filters(model.keras_model.layers[layer_idx])
        filter_indices = sorted(np.random.permutation(n)[:max_filters])

    selected_indices.append(filter_indices)

    print("Layer {} has {} fitlers, filter {} is selected.".format(layer_name, n, filter_indices))

    # Generate input image for each filter.
    for idx in filter_indices:
        img = visualize_activation(model.keras_model, layer_idx, filter_indices=idx, tv_weight=0., input_modifiers=[Jitter(0.05)], max_iter=300) 
        vis_images[i].append(img)


    # Generate stitched image palette with 4 cols so we get 2 rows.
    stitched = utils.stitch_images(vis_images[i], cols=4)    
    plt.figure(figsize=(20, 30))
    plt.title(layer_name)
    plt.axis('off')
    plt.imshow(stitched)
    plt.show()
    i += 1

In [ ]:
new_vis_images = [[], [], [], [], []]
i = 0
for layer_name in conv_layer_names:
    layer_idx = utils.find_layer_idx(model.keras_model, layer_name)

    # Generate input image for each filter.
    for j, idx in enumerate(selected_indices[i]):
        img = visualize_activation(model.keras_model, layer_idx, filter_indices=idx, seed_input=vis_images[i][j], input_modifiers=[Jitter(0.05)]) 
        img = utils.draw_text(img, 'Filter {}'.format(idx))  
        new_vis_images[i].append(img)

    stitched = utils.stitch_images(new_vis_images[i], cols=4)    
    plt.figure(figsize=(20, 30))
    plt.title(layer_name)
    plt.axis('off')
    plt.imshow(stitched)
    plt.show()
    i += 1